In [0]:
!cp -r /content/drive/My\ Drive/cut /content

In [2]:
% cd /content/drive/My Drive/Colab Notebooks/csce636/course_project
!pip install jsonlines
% ls

/content/drive/My Drive/Colab Notebooks/csce636/course_project
 ava_train_v2.2.csv       model1.4_02-0.62.hdf5   model3_30-0.44.hdf5
 ava_val_v2.2.csv         model1.4_07-0.61.hdf5   model3_43-0.44.hdf5
'Copy of model_4.ipynb'   model1.4_08-0.59.hdf5   model3_44-0.37.hdf5
 log/                     model1.4_11-0.57.hdf5   model_4.ipynb
 model_01-0.55.hdf5       model1.4_13-0.43.hdf5   model_final.ipynb
 model_01-0.63.hdf5       model1.4_18-0.33.hdf5   model_l_01-0.61.hdf5
 model_02-0.56.hdf5       model1.5_01-0.68.hdf5   model_l_04-0.34.hdf5
 model_03-0.46.hdf5       model1.5_03-0.62.hdf5   model_l1.2_01-0.72.hdf5
 model_05-0.43.hdf5       model1.5_09-0.54.hdf5   model_l1.2_04-0.51.hdf5
 model_05-0.49.hdf5       model1.5_10-0.46.hdf5   model_l1.2_07-0.48.hdf5
 model1.2_01-0.72.hdf5    model1.5_28-0.37.hdf5   model_l1.2_15-0.30.hdf5
 model1.2_03-0.68.hdf5    model_23-0.31.hdf5      model_l1.3_01-0.56.hdf5
 model1.2_05-0.65.hdf5    model3_01-1.42.hdf5     model_l1.3_01-0.78.hdf5
 model1.2

In [3]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics
from keras.optimizers import SGD
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', 
                         include_top=False,
                         input_shape=(299,299,3),
                         pooling=None
                         )





from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, GlobalAveragePooling2D, Flatten
# add a global spatial average pooling layer



x2 = base_model.output
x2 = GlobalAveragePooling2D()(x2)
predictions2 = Dense(1, activation='sigmoid')(x2)

# this is the model we will train
model2 = Model(inputs=base_model.input, outputs=predictions2)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in model2.layers[:249]:
   layer.trainable = False
for layer in model2.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model2.compile(optimizer=SGD(lr=0.001, momentum=0.9),
              loss='binary_crossentropy',
              metrics=['acc'])
#model.compile(loss=losses.binary_crossentropy,
            #optimizer=optimizers.RMSprop(lr=0.0001),
            #metrics=[metrics.binary_accuracy])

model2.load_weights('model_l1.3_24-0.28.hdf5')#/content/drive/My Drive/Colab Notebooks/csce636/course_project/model_l1.3_24-0.28.hdf5

Using TensorFlow backend.


87916544/87910968 [==============================] - 1s 0us/step


In [7]:
import os, shutil
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
test_dir='/content/cut'


test_generator = datagen.flow_from_directory(
    directory=test_dir,
    target_size=(299, 299),
    color_mode="rgb",
    batch_size=64,
    class_mode="binary",
    shuffle=True,
    seed=42
)


Found 2257 images belonging to 2 classes.


In [0]:
results=model2.evaluate_generator(test_generator, callbacks=None,  
                    use_multiprocessing=False, verbose=0)

In [9]:
results

[0.8451483249664307, 0.5454142689704895]

In [0]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=optimizers.RMSprop(lr=0.0001), loss='binary_crossentropy', 
              metrics=['acc'])
#model.compile(loss=losses.binary_crossentropy,
            #optimizer=optimizers.RMSprop(lr=0.0001),
            #metrics=[metrics.binary_accuracy])
model.load_weights('model1.5_28-0.37.hdf5')

In [0]:
results=model.evaluate_generator(test_generator, callbacks=None,  
                    use_multiprocessing=False, verbose=0)

In [14]:
results

[1.155958652496338, 0.4319893717765808]